## Script for the calculation of the normalization constant


In this script, I calculate the normalization constant we have to use in order to get the same flux above $0.1 GeV$ in the following two cases: a Dark Matter particle with mass $m_{dm} = 30$GeV and $m_{dm} =10$GeV. 

The flux above $0.1$GeV is:

\begin{equation}
F_i = \int_{0.1 GeV}^{m_{dm,i}} \frac{dN}{dE}dE = \int_{\log 0.1 GeV}^{\log m_{dm,i}} \frac{dN}{d \log E}d\log E, \qquad m_{dm,i} = 10 GeV,\, 30 GeV.
\end{equation}


Then, what i'm going to do, after calculating $F_{10},\, F_{30}$ is to find the ratio $\frac{F_{30}}{F_{10}}$. This will serve as our normilzation factor: I will first compute the number of events per energy bin for the case of a $10$GeV dark matter exactly as I did for the case of the $30$GeV dark matter (including the normalization factor we used there) but I will also multiply the final result by that normalization factor ${\cal{N}} \equiv F_{30}/F_{10}$.


So, let's calculte 'em!!


In [20]:
import numpy as np 

#We open and read the file containing the spectrum. The file should be at the same directory as our script/notebook
mass, energ, bibibar = np.loadtxt("AtProduction_gammas.dat", usecols=(0,1,13), unpack=3)


#since we don't know the size of the file, let's find the length of the columns. Find the upper limit; 
#when then we are going to run a for loop, we will know the range
a = np.shape(mass)
uplim=a[0]-1


#From now on we will need to play with two different masses. Namely, 10GeV and 30 GeV. We will denote everything related 
#the 30 GeV dark matter by the name of the variable followed by the number 30 and everything related to the 10 GeV by the 
#name of the variable followed by - guess what: 10!

#So, dark matter masses:

mdm10 = 10.0
mdm30 = 30.0

#Now, let us find how many entries have each of these values. We will need this number in a while.

size10 = 0
for i in range (0,uplim):
    if (mass[i] == mdm10):
        size10 +=1


size30 = 0
for i in range (0,uplim):
    if (mass[i] == mdm30):
        size30 +=1



#size is the number of entries corresponding to the specific value of mass. Let us create two arrays with zeros, of that size.
# The titles are logex and diendlogex, which are quite descriptive about what they contain (what they are going to contain).

logex10 = np.zeros(size10)
diendlogex10 = np.zeros(size10)

logex30 = np.zeros(size30)
diendlogex30 = np.zeros(size30)


#Let's populate the arrays logex, diendlogex
# x=photon energy / dark matter particle mass

j = 0 
for i in range (0,uplim):
    if (mass[i] == mdm10):
        logex10[j] = energ[i]
        diendlogex10[j] = bibibar[i]
        j +=1

j = 0 
for i in range (0,uplim):
    if (mass[i] == mdm30):
        logex30[j] = energ[i]
        diendlogex30[j] = bibibar[i]
        j +=1





In [21]:
#Remember, each array starts from zero. Useful to define the number size-1, call it n, which will serve as an upper limit

n10 = size10 - 1 
n30 = size30 - 1

#just useful when we want to find the entry with the maximum index
#If we generally want to run a loop, limits 0,size are the approptiate

Now: $\log_{10}x = \log_{10} \frac{E}{M_{dm}} = \log_{10} \frac{E}{GeV} \frac{GeV}{M_{dm}} \Rightarrow \log_{10} \frac{E}{GeV} = \log_{10}x + \log_{10} \frac{M_{dm}}{GeV} $. Since everything is expressed in GeV, is easy to create the array  
containing the logs of the energies:

In [22]:
logE10 = logex10 + np.log10(mdm10)
logE30 = logex30 + np.log10(mdm30)

The other one does not require adjustment. $dN/d \log_{10} x = \ln10 x dN/dx = \ln10 \,E \, \frac{dN}{dE} = \frac{dN}{d\log E}$. So, remain as it is. 



In [23]:
dNdlogE10 = diendlogex10
dNdlogE30 = diendlogex30

In [24]:
#Starting energy of photons in GeV - 0.1 in both cases, so we do not need to distinguish here between the two cases

#Remember again that we have  

Est = 0.1


#We want to calculate the integral of the flux - see formula above - in the two cases 

#Let's now use interpolation in order to have a value corresponding to dN/dlogE for every value of energy

from scipy import interpolate

intrpdNdlogE10 = interpolate.interp1d(logE10, dNdlogE10)
intrpdNdlogE30 = interpolate.interp1d(logE30, dNdlogE30)


#Calculate the upper and lower limits of the two intergrals of the flux - for both cases 

low10 = np.log10(Est)
low30 = np.log10(Est)
up10 = np.log10(mdm10)
up30 = np.log10(mdm30)



def integralino10(a,b):
    #Number of points
    nu = 10000
    #step 
    h = (b - a)/nu
    summa = 0.0
    for j in range (0,nu):
        x1 = a + j*h
        x2 = x1 + h
        q1 = intrpdNdlogE10(x1)
        q2 = intrpdNdlogE10(x2)
        summa = summa +  q1 + q2
    summa = summa*h/2.0
    return summa



def integralino30(a,b):
    #Number of points
    nu = 10000
    #step 
    h = (b - a)/nu
    summa = 0.0
    for j in range (0,nu):
        x1 = a + j*h
        x2 = x1 + h
        q1 = intrpdNdlogE30(x1)
        q2 = intrpdNdlogE30(x2)
        summa = summa +  q1 + q2
    summa = summa*h/2.0
    return summa






#Calculation of fluxes 

#Flux  over 0.1 GeV in the case of 10 GeV dark matter particle 
F10 = integralino10(low10, up10)


#Flux over 0.1 GeV in the case of 30 GeV dark matter particle
F30 = integralino30(low30, up30)



Normalization constant defined, as we said as:

\begin{equation}
{\cal{N}} \equiv \frac{F_{30}}{F_{10}}
\end{equation}

In [25]:
Norm = F30/F10

print("Normalization constant is equal to:")
print(Norm)

Normalization constant is equal to:
1.67383318364
